In [4]:
import pandas as pd
from pandas import DataFrame
import csv
import numpy as np
from gensim.models.word2vec import Word2Vec


## load file for attributes
attribute_file = "./source/lib/attribute.csv"
f = open(attribute_file, "r", encoding = "utf-8")
data = csv.reader(f)
f.close
next(data)
attributes = []
for row in data:
    if row[-1]:
        attributes.append(row[0])

        
## load models for comparison       
comparison_file = "./source/lib/brands.csv"
f = open(comparison_file, "r", encoding = "utf-8")
data = csv.reader(f)
f.close
next(data)
brands = []
for row in data:
    # 브랜드라면 brands에 추가
    if row[-1]:
        brands.append(row[0])

        
##setting criteria brand and coefficient        
comp_model_file ="./source/model/word2vec_치킨_total.model"
comp_model = Word2Vec.load(comp_model_file)        

bbrand = input("기준 브랜드를 입력하세요 : ")
criteria_brand = [bbrand]
criteria_attributes = []

root_model = "./source/model/word2vec_라면_total.model"
model = Word2Vec.load(root_model)

## 기준치 이상의 attribute만 append
for attribute in attributes:
    if model.wv.similarity(criteria_brand, attribute)[0] > 0.5:
        criteria_attributes.append(attribute)

##계수 구축
coeff = pd.DataFrame()
coeff = pd.DataFrame(coeff, columns = criteria_attributes, index = criteria_brand)
for c_attribute in criteria_attributes:
    buff = float(np.round(((model.wv.similarity(criteria_brand, c_attribute) + 1) / 2), 5))
    coeff[c_attribute] = buff

    
## calculating ratings    
rating = pd.DataFrame()
rating = pd.DataFrame(rating, columns = criteria_attributes, index = brands)

## attribute와 brands의 similarity와 coeff의 곱을 rating에 저장
for c_attribute in criteria_attributes:
    for brand in brands:
        rating.loc[brand, c_attribute] = round((((comp_model.wv.similarity(brand, c_attribute) + 1) / 2 ) * coeff.loc[criteria_brand[0], c_attribute]), 5)

#total of columns 추가(10점 만점 변환)
rating["total"] = (rating.sum(axis = 1))/len(criteria_attributes) * 10


display(rating)

기준 브랜드를 입력하세요 : 진라면


,저렴하다,맵다,유명하다,고급스럽다,total
뿌링클,0.61175,0.55883,0.67486,0.61988,6.163300
레드콤보,0.58285,0.56644,0.66105,0.59900,6.023350
황금올리브,0.59405,0.55011,0.65153,0.60433,6.000050
푸라닭,0.59815,0.57896,0.66769,0.59603,6.102075
호식이,0.55911,0.54270,0.62753,0.59151,5.802125
